# ESML: Customize Scoring script: 
## Redeploy model  on AKS, adding predict_proba to scoring_script (based on AutoML generated)
- 1) `AutoMap datalake` & init ESML project
- 2) `Get earlier trained model`
- 3) `Get AutoML genereated scoring script`
    - This DEMO notebook, uses TITANIC, to add predict_proba.
    - Copy from here: `\azure-enterprise-scale-ml\copy_my_subfolders_to_my_grandparent\settings\project_specific\model\dev_test_prod\train\automl\scoring_file_dev_M01_titanic.py`
    - Paste to YOUR settings folder: `\settings\project_specific\model\dev_test_prod\train\automl\scoring_file_dev_M01_titanic.py`
- 4) `Customize scoring script & redeploy`




######  NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLDataset, ESMLProject

p = ESMLProject()
p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
######  NB!

In [3]:
unregister_all_datasets=False

In [4]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLDataset, ESMLProject

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../../../settings', you should copy template settings from '../settings'
p.inference_mode = False # We want "TRAIN" mode
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace

if(unregister_all_datasets):
    p.unregister_all_datasets(ws) # For DEMO purpose
    

In [5]:
p.describe()

Inference version: 1

 - ds01_titanic
projects/project002/01_titanic_model_clas/train/ds01_titanic/in/dev/2021/01/01/
projects/project002/01_titanic_model_clas/train/ds01_titanic/out/bronze/dev/
projects/project002/01_titanic_model_clas/train/ds01_titanic/out/silver/dev/
projects/project002/01_titanic_model_clas/inference/1/ds01_titanic/in/dev/2021/01/01/
 

Training GOLD
projects/project002/01_titanic_model_clas/train/gold/dev/
 

A)INFERENCE ONLINE: GOLD to score (example if realtime - today)
projects/project002/01_titanic_model_clas/inference/1/gold/dev/2021_06_16/066b4fd80ee746e1b3e7a4a6e0300ba5/
 

A)INFERENCE ONLINE: GOLD scored (example if realtime today)
projects/project002/01_titanic_model_clas/inference/1/scored/dev/2021_06_16/066b4fd80ee746e1b3e7a4a6e0300ba5/
 

B)INFERENCE BATCH: GOLD to score (example batch, datetime from config)
projects/project002/01_titanic_model_clas/inference/1/gold/dev/2021_01_01/3ed03414d8444f40a764cd1a8f311f90/
 

B)INFERENCE BATCH: GOLD scored (ex

In [4]:
datastore = p.init() # 3) Automapping from datalake to Azure ML datasets

...
Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = TRUE 
 - Found settings in the ESML AutoLake  [active_in_folder.json,active_scoring_in_folder.json], to override ArgParse/GIT config with.
 - TRAIN in date:  2021/01/01
 - INFERENCE in date: 2021/06/08 and ModelVersion to score with: 1 (0=latest)

Load data as Datasets....
ds01_diabetes

####### Automap & Autoregister - SUCCESS!
1) Auto mapped 1 ESML Dataset with registered Azure ML Datasets (potentially all 3: IN,BRONZE, SILVER) in Datastore project002 

Dataset 'ds01_diabetes' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files

2) Registered each Dataset with suffixes (_IN, _BRONZE, _SILVER) 
 Tip: Use ESMLProject.Datasets list or .DatasetByName(myDatasetName) to read/write
#######


train, evauluate, test = p.split_gold_3(0.6,label)

# 2) Azure ML `Edit AutoML generated scoring_script`
 - Path: `project_specific\\model\\dev_test_prod\\train\\automl\\scoring_file_dev.py`
 - DEMO custom script Path: `project_specific\\model\\dev_test_prod\\train\\automl\\scoring_file_custom.py`

In [12]:
#2a - Download from Azure
inference_config_to_override_and_inject, model, best_run = p.get_active_model_inference_config() # 1) You can override this scoring_script - get a baseline, then modify...

Loading AutoML config settings from: dev
Loading AutoML config settings from: dev
Package:azureml-automl-runtime, training version:1.29.0, current version:1.26.0
Package:azureml-core, training version:1.29.0, current version:1.26.0
Package:azureml-dataprep, training version:2.15.1, current version:2.13.2
Package:azureml-dataprep-native, training version:33.0.0, current version:32.0.0
Package:azureml-dataprep-rslex, training version:1.13.0, current version:1.11.2
Package:azureml-dataset-runtime, training version:1.29.0, current version:1.26.0
Package:azureml-defaults, training version:1.29.0, current version:1.26.0
Package:azureml-interpret, training version:1.29.0, current version:1.26.0
Package:azureml-pipeline-core, training version:1.29.0, current version:1.26.0
Package:azureml-telemetry, training version:1.29.0, current version:1.26.0
Package:azureml-train-automl-client, training version:1.29.0, current version:1.26.0
Package:azureml-train-automl-runtime, training version:1.29.0, c

In [13]:
inference_config_to_override_and_inject.entry_script # Shows the scoring_script PATH, downloaded locally, ready to be edited and overridden

'c:\\Users\\jostrm\\OneDrive - Microsoft\\0_GIT\\2_My\\github\\azure-enterprise-scale-ml\\settings\\project_specific\\model\\dev_test_prod\\train\\automl\\scoring_file_dev.py'

In [14]:
#2b - Copy file, and edit, your CUSTOM scoring_script - see example "scoring_file_custom.py" ...and overwrite .entry_script

my_custom_script_instead = 'scoring_file_custom.py'
inference_config_to_override_and_inject.entry_script = 'c:\\Users\\jostrm\\OneDrive - Microsoft\\0_GIT\\2_My\\github\\azure-enterprise-scale-ml\\settings\\project_specific\\model\\dev_test_prod\\train\\automl\\' + my_custom_script_instead

inference_config_to_override_and_inject.entry_script # Verify path

'c:\\Users\\jostrm\\OneDrive - Microsoft\\0_GIT\\2_My\\github\\azure-enterprise-scale-ml\\settings\\project_specific\\model\\dev_test_prod\\train\\automl\\scoring_file_custom.py'

# 3) ESML `Deploy model ONLINE` in `1 line of code` (AKS) 
- Deploy "offline" MODEL from old `run` in environment To →  `DEV`, `TEST` or `PROD` environment
- ESML saves `API_key in Azure keyvault automatically`
- ESML auto-config solves 4 common 'errors/things': `correct compute name` and `valid replicas, valid agents, valid auto scaling`
    - Tip: You can adjust the number of replicas, and different CPU/memory configuration, or using a different compute target.

In [15]:
# DEPLOY with custom InferenceConfig (custom scoring script)
service,api_uri, kv_aks_api_secret= p.deploy_automl_model_to_aks(model,inference_config_to_override_and_inject, True) #2) (model,inference_config, overwrite_endpoint=True,deployment_config=None):

Deploying model: AutoML7d5ab21270 with verison: 1 to environment: dev with overwrite_endpoint=True
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
image_build_compute = prj02-m03-dev
Found existing AksWebservice endpoint, deleting it, since overwrite=True
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster, esml-dev-prj02, using it.
Note: Autoscale_enabled=False, or since aks_dev_test=True in config, autoscaling is automatically shut off, e.g. overridden in config (since not supported) for environment dev
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-0

## 3b) ESML Test AKS webservice, `2 lines of code`


esml_dataset_bronze_df = p.DatasetByName("ds01_titanic").Bronze.to_pandas_dataframe() 
gold_train = p.save_gold(esml_dataset_bronze_df)  
label = "Survived"
train_set, validate_set, test_set = p.split_gold_3(0.6,label)
X_test, y_test, tags = p.get_gold_validate_Xy() 


In [24]:
label = "Survived"
to_score = None
try:
    X_test = p.GoldTest.to_pandas_dataframe()
    to_score = X_test.drop([label], axis=1)
    print(to_score.head()) # gold_test_1 = Dataset.get_by_name(ws, name=p.dataset_gold_test_name_azure)
except: 
    print ("you need to have splitted GOLD dataset, GoldTest need to exist. Change next cell from MARKDOWN, to CODE, and run that. Try this again... ")

   Pclass                                               Name     Sex   Age  \
0       3                           Miss. Bridget O'Driscoll  female  27.0   
1       2  Mrs. William John Robert (Dorothy Ann Wonnacot...  female  27.0   
2       2                               Master. Andre Mallet    male   1.0   
3       1                 Col. Oberst Alfons Simonius-Blumer    male  56.0   
4       1                        Miss. Jean Gertrude Hippach  female  16.0   

   Siblings/Spouses Aboard  Parents/Children Aboard     Fare  
0                        0                        0   7.7500  
1                        1                        0  21.0000  
2                        0                        2  37.0042  
3                        0                        0  35.5000  
4                        0                        1  57.9792  


In [34]:
from baselayer_azure_ml import ComputeFactory
import json
keyvault = p.ws.get_default_keyvault()
api_uri = keyvault.get_secret(name='esml-dev-p02-m01-api') # titanic = m01
api_key = keyvault.get_secret(name='esml-dev-p02-m01-apisecret') #

result_json = ComputeFactory.call_webservice_static(to_score, api_uri,api_key,firstRowOnly=False) # Simulate "REST call" (no ESML dependancy)
res_dict = json.loads(result_json.text) # json -> dictionary
df_res = pd.read_json(res_dict) # dictionary -> pandas
all_result = X_test.join(df_res) # features + result
all_result

Relying on you having the keys...


,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,result,probability
0,1,3,Miss. Bridget O'Driscoll,female,27.00,0,0,7.7500,1,0.922252
1,0,2,Mrs. William John Robert (Dorothy Ann Wonnacot...,female,27.00,1,0,21.0000,0,0.234340
2,1,2,Master. Andre Mallet,male,1.00,0,2,37.0042,1,0.984918
3,1,1,Col. Oberst Alfons Simonius-Blumer,male,56.00,0,0,35.5000,0,0.338251
4,1,1,Miss. Jean Gertrude Hippach,female,16.00,0,1,57.9792,1,0.999310
...,...,...,...,...,...,...,...,...,...,...
528,1,2,Master. Viljo Hamalainen,male,0.67,1,1,14.5000,1,0.996252
529,1,1,Mr. Henry Blank,male,40.00,0,0,31.0000,0,0.377782
530,0,3,Mr. Sleiman Attalah,male,30.00,0,0,7.2250,0,0.013294
531,0,3,Miss. Ida Lefebre,female,3.00,3,1,25.4667,0,0.032332


### Alt 2 - use compute factory, control to `get JSON back` instead of PANDAS. 
#### `No saving to LAKE`

In [35]:
result, model_version_used = p.compute_factory.call_webservice(to_score,False,False) # (X_test, firstRowOnly=True,pandas_result=True, api_uri=None,api_key="auto from keyvault")
df_res = pd.read_json(result)
all_result = to_score.join(df_res) # Need to join the FEATURES yourself, post webservice call (simulate no ESML dependancy in caller)
all_result

Note: Fetching keys automatically via workspace keyvault.


,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,result,probability
0,3,Miss. Bridget O'Driscoll,female,27.00,0,0,7.7500,1,0.922252
1,2,Mrs. William John Robert (Dorothy Ann Wonnacot...,female,27.00,1,0,21.0000,0,0.234340
2,2,Master. Andre Mallet,male,1.00,0,2,37.0042,1,0.984918
3,1,Col. Oberst Alfons Simonius-Blumer,male,56.00,0,0,35.5000,0,0.338251
4,1,Miss. Jean Gertrude Hippach,female,16.00,0,1,57.9792,1,0.999310
...,...,...,...,...,...,...,...,...,...
528,2,Master. Viljo Hamalainen,male,0.67,1,1,14.5000,1,0.996252
529,1,Mr. Henry Blank,male,40.00,0,0,31.0000,0,0.377782
530,3,Mr. Sleiman Attalah,male,30.00,0,0,7.2250,0,0.013294
531,3,Miss. Ida Lefebre,female,3.00,3,1,25.4667,0,0.032332


### Alt 3 - ESML.call_webservice, `get PANDAS joined` dataframe
#### `Also saves to LAKE, automatically` 
- Requires p.init() or to manually set lake `p.lakestore = p.set_lake_as_datastore(p.ws)`

In [36]:
p.lakestore = p.set_lake_as_datastore(p.ws) # only needed if NOT p.init() is done
df = p.call_webservice(p.ws, to_score,"caller_id") # (X_test, firstRowOnly=True,pandas_result=True, api_uri=None,api_key="auto from keyvault")
df.head()

Using GEN2 as Datastore
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Note: Fetching keys automatically via workspace keyvault.
Saving scoring to lake for project folder project002 and inference_model_version: 2 ...
...

Saved DATA to score successfully in LAKE, as file 'to_score_caller_id.parquet'
Saved SCORED data in LAKE, as file 'scored_caller_id.parquet'


,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,result,probability
0,3,Miss. Bridget O'Driscoll,female,27.0,0,0,7.7500,1,0.922252
1,2,Mrs. William John Robert (Dorothy Ann Wonnacot...,female,27.0,1,0,21.0000,0,0.234340
2,2,Master. Andre Mallet,male,1.0,0,2,37.0042,1,0.984918
3,1,Col. Oberst Alfons Simonius-Blumer,male,56.0,0,0,35.5000,0,0.338251
4,1,Miss. Jean Gertrude Hippach,female,16.0,0,1,57.9792,1,0.999310


## Optional: You can ALSO customize DeployConfig for AKS, programmatically...
- Can be done, `BUT recommendation is to use the "settings/" folder instead`, to override performance settings in .json per each evnironment (dev,test,prod)

In [37]:
deploy_config_to_override_and_inject = p.get_deploy_config_aks()  # 2) Optional: you can also override this - get a baseline, then modify...

print("description:", deploy_config_to_override_and_inject.description)
print("tags: ", deploy_config_to_override_and_inject.tags)
print("config: ", deploy_config_to_override_and_inject.print_deploy_configuration())
print("config - cpu_cores_limit (override what you want): ", deploy_config_to_override_and_inject.cpu_cores_limit)

Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Note: Autoscale_enabled=False, or since aks_dev_test=True in config, autoscaling is automatically shut off, e.g. overridden in config (since not supported) for environment dev
description: AKS webserice in DEV environment.
tags:  {'environment': 'dev'}
CPU requirement: 1, Memory requirement: 8GB, Number of replica: 1
config:  None
config - cpu_cores_limit (override what you want):  None


In [ ]:
# DEPLOY with custom DeployConfig
service,api_uri, kv_aks_api_secret= p.deploy_automl_model_to_aks(model,inference_config_to_override_and_inject, True, own_deploy_config_to_inject) 

In [13]:
inference_config, model, best_run = p.get_active_model_inference_config(ws) #  AutoML support 
service,api_uri, kv_aks_api_secret= p.deploy_automl_model_to_aks(model,inference_config,True) #2) (model,inference_config, overwrite_endpoint=True,deployment_config=None):

Loading AutoML config settings from: dev
Loading AutoML config settings from: dev
Deploying model: AutoML449305f300 with verison: 1 to environment: dev with overwrite_endpoint=True
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing AksWebservice endpoint, deleting it, since overwrite=True
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster, esml-dev-prj02, using it.
Note: Autoscale_enabled=False, or since aks_dev_test=True in config, autoscaling is automatically shut off, e.g. overridden in config (since not supported) for environment dev
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deploymen

# END